In [1]:
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
 
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, utils
import pickle
import numpy as np
import torch
import torchvision
import torchvision.transforms.functional as TF
import pandas as pd
from ast import literal_eval
import glob
import os
import numpy as np
from PIL import Image, ImageDraw
import cv2
import time
import pandas as pd

In [2]:
class myDataset(torch.utils.data.Dataset):
    def __init__(self, args, dtype):
        
        self.args = args
        self.dtype = dtype
        print("Loading",self.dtype)
        
#         sequence_centric = pd.read_csv("sequences_16_overlap_4_thres0_"+self.dtype+".csv")
        sequence_centric = pd.read_csv("sequences_4_"+self.dtype+".csv")

        df = sequence_centric.copy()      
        for v in list(df.columns.values):
            print(v+' loaded')
            try:
                df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
            except:
                continue
        sequence_centric[df.columns] = df[df.columns]
        self.data = sequence_centric.copy().reset_index(drop=True)
    
        print('*'*30)
        

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index):

        seq = self.data.iloc[index]
        outputs = []

        observed = torch.tensor(np.array(seq.Pose))
        future = torch.tensor(np.array(seq.Future_Pose))

        obs = torch.tensor([seq.Pose[i] for i in range(0,self.args.input,self.args.skip)])
        obs_speed = (obs[1:] - obs[:-1])
    
        outputs.append(obs_speed)
        
        
        true = torch.tensor([seq.Future_Pose[i] for i in range(0,self.args.output,self.args.skip)])
        true_speed = torch.cat(((true[0]-obs[-1]).unsqueeze(0), true[1:]-true[:-1]))


        outputs.append(true_speed)
        outputs.append(obs)
        outputs.append(true)
        
        return tuple(outputs)    
    
    
def data_loader(args,data):
    dataset = myDataset(args,data)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size, shuffle=args.loader_shuffle,
        pin_memory=args.pin_memory)

    return dataloader



#Interpolation

In [19]:
seq_len=4
batch_size=2
def ADE_c(pred, true):
#     print(pred[0,0])
    pred = torch.reshape(pred, (batch_size,seq_len,25,2))
    true = torch.reshape(true, (batch_size,seq_len,25,2))
    
    t=torch.sqrt((pred[:,:,:,0]-true[:,:,:,0])**2+(pred[:,:,:,1]-true[:,:,:,1])**2)
    print(t.shape)
#     print(pred[0,0])
#     print(pred.shape,true.shape)
#     displacement = torch.sqrt(torch.sum((pred-true])**2,dim=0))
#     ade = torch.mean(displacement)
    
#     return ade


def FDE_c(pred, true):
    displacement = torch.sqrt(torch.sum((pred[:,-1,:]-true[:,-1,:])**2,dim=0))
    fde = torch.mean(displacement)
    
    return fde

def speed2pos(preds, obs_p):
    pred_pos = torch.zeros(preds.shape[0], preds.shape[1], 50)
    current = obs_p[:,-1,:]
    
    for i in range(preds.shape[1]):
        pred_pos[:,i,:] = current + preds[:,i,:]
        current = pred_pos[:,i,:]
        
    for i in range(50):
        pred_pos[:,:,i] = torch.min(pred_pos[:,:,i], 1920*torch.ones(pred_pos.shape[0], pred_pos.shape[1]))#, device='cuda'))
    #     pred_pos[:,:,1] = torch.min(pred_pos[:,:,1], 1080*torch.ones(pred_pos.shape[0], pred_pos.shape[1]))#, device='cuda'))
        pred_pos[:,:,i] = torch.max(pred_pos[:,:,i], torch.zeros(pred_pos.shape[0], pred_pos.shape[1]))#, device='cuda'))
    #     pred_pos[:,:,1] = torch.max(pred_pos[:,:,1], torch.zeros(pred_pos.shape[0], pred_pos.shape[1]))#, device='cuda'))
        
    return pred_pos

In [20]:
class args():
    def __init__(self):
        self.loader_workers = 1
        self.loader_shuffle = False
        self.device         = 'cuda'
        self.batch_size     = 2
        self.n_epochs       = 200
        self.hidden_size    = 6400
        self.hardtanh_limit = 100
        self.input  = 4
        self.output = 4
        self.pin_memory= False
        self.stride = 4
        self.skip   = 1
        self.task   = 'pose'
        self.use_scenes = False       
        self.lr = 0.001
        
args = args()
val_loader=data_loader(args,"val" )
for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(val_loader):
#     print(obs_s)
#     print(target_s)
#     print(obs_pose)
    print(target_pose)
    
    preds_p=speed2pos(target_s,obs_pose)
    print(preds_p)
    print(float(ADE_c(preds_p, target_pose)))
    print(float(FDE_c(preds_p, target_pose)))
            
#         for i in range(1,16):
#             preds_p = speed_preds[:,i,:]+preds_p[:,-1:,:]
#             ade_train += float(ADE_c(preds_p, target_pose[:,i:i+1,:]))
#             fde_train += float(FDE_c(preds_p, target_pose[:,i:i+1,:]))
            
        
    
    break

Loading val
Pose loaded
Future_Pose loaded
******************************
tensor([[[1139.,  677., 1142.,  678., 1142.,  680., 1143.,  692., 1145.,  701.,
          1143.,  677., 1148.,  695., 1145.,  704., 1142.,  707., 1142.,  707.,
          1140.,  725., 1139.,  751., 1142.,  707., 1142.,  727., 1140.,  751.,
          1140.,  674., 1140.,  672., 1140.,  672., 1142.,  672., 1143.,  754.,
          1142.,  751., 1140.,  754., 1140.,  754., 1137.,  754., 1137.,  754.],
         [1139.,  671., 1142.,  672., 1143.,  672., 1148.,  692., 1145.,  698.,
          1140.,  672., 1140.,  692., 1142.,  698., 1140.,  704., 1142.,  704.,
          1140.,  725., 1140.,  745., 1140.,  704., 1140.,  725., 1140.,  745.,
          1142.,  669., 1139.,  669., 1142.,  663., 1140.,  669., 1143.,  751.,
          1143.,  751., 1142.,  748., 1142.,  748., 1142.,  751., 1140.,  745.],
         [1140.,  669., 1142.,  672., 1142.,  672., 1142.,  692., 1140.,  695.,
          1142.,  672., 1143.,  692., 1145.,

TypeError: float() argument must be a string or a number, not 'NoneType'

# PoseLSTM

In [25]:
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
 
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, utils
import pickle
import numpy as np
import torch
import torchvision
import torchvision.transforms.functional as TF
import pandas as pd
from ast import literal_eval
import glob
import os
import numpy as np
from PIL import Image, ImageDraw
import cv2
import time
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms


class PV_LSTM_DE_pose(nn.Module):
    def __init__(self, args):
        '''
           input: observed body poses and velocites global and local
           output: global and local velocities
        '''
        super(PV_LSTM_DE_pose, self).__init__()
         
        self.pose_encoder = nn.LSTM(input_size=34, hidden_size=args.hidden_size)
        self.pose_embedding = nn.Sequential(nn.Linear(in_features=args.hidden_size, out_features=34),
                                           nn.ReLU())
        self.pose_decoder = nn.LSTMCell(input_size=34, hidden_size=args.hidden_size)
        self.fc_pose   = nn.Linear(in_features=args.hidden_size, out_features=34)
        
        self.vel_encoder = nn.LSTM(input_size=34, hidden_size=args.hidden_size)
        self.vel_decoder = nn.LSTMCell(input_size=34, hidden_size=args.hidden_size)
        self.fc_vel    = nn.Linear(in_features=args.hidden_size, out_features=34)
        
        self.pose_glob_encoder = nn.LSTM(input_size=2, hidden_size=100)
        self.vel_glob_encoder = nn.LSTM(input_size=2, hidden_size=100)
        self.vel_glob_decoder = nn.LSTMCell(input_size=2, hidden_size=100)
        self.fc_global    = nn.Linear(in_features=100, out_features=2)
       
        
        self.hardtanh = nn.Hardtanh(min_val=-1*args.hardtanh_limit,max_val=args.hardtanh_limit)
        self.relu = nn.ReLU() 
        self.softmax = nn.Softmax(dim=1)
        
        
        self.args = args
        
    def forward(self, pose=None, vel=None, pose_glob=None, vel_glob=None):


        _, (hidden_vel, cell_vel) = self.vel_encoder(vel.permute(1,0,2))
        hidden_vel = hidden_vel.squeeze(0)
        cell_vel = cell_vel.squeeze(0)


        _, (hidden_pose, cell_pose) = self.pose_encoder(pose.permute(1,0,2))
        hidden_pose = hidden_pose.squeeze(0)
        cell_pose = cell_pose.squeeze(0)
        
        outputs = []
        
      
        #predicting the velocity of poses and poses
        vel_outputs    = torch.tensor([], device=self.args.device)
        pose_outputs   = torch.tensor([], device=self.args.device)
        glob_outputs    = torch.tensor([], device=self.args.device)

        VelDec_inp = vel[:,-1,:]
        PoseDec_inp = pose[:,-1,:]
        
        hidden_dec = hidden_pose + hidden_vel
        cell_dec = cell_pose + cell_vel
         
#         outputs.append(vel_outputs)
        for i in range(self.args.output//self.args.skip):
            hidden_dec, cell_dec = self.vel_decoder(VelDec_inp, (hidden_dec, cell_dec))
            vel_output  = self.hardtanh(self.fc_vel(hidden_dec))
#             vel_outputs = torch.cat((vel_outputs, vel_output.unsqueeze(1)), dim = 1)
            
            pose_output = PoseDec_inp + VelDec_inp
            pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
            PoseDec_inp = pose_output
            VelDec_inp  = vel_output
                   
        outputs.append(pose_outputs)
#         in_pose = pose[:,-1,:]
          
#         hdp = hidden_pose + hidden_vel
#         cdp = cell_pose + cell_vel
#         for i in range(self.args.output//self.args.skip):
#             hdp, cdp     = self.pose_decoder(in_pose, (hdp, cdp))
#             pose_output  = self.fc_pose(hdp)
# #             in_pose      = self.pose_embedding(hdp).detach()
# #             pose_output  = self.softmax(pose_output)
# #             pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
#             pose_output = PoseDec_inp + VelDec_inp 
#             in_pose  = pose_output
            
#             pose_outputs = torch.cat((pose_outputs, pose_output.unsqueeze(1)), dim = 1)
# #             PoseDec_inp = pose_output

#         outputs.append(pose_outputs)
    
        
        
#         _, (hidden_vel_glob, cell_vel_glob) = self.vel_glob_encoder(vel_glob.permute(1,0,2))
#         hidden_vel_glob = hidden_vel_glob.squeeze(0)
#         cell_vel_glob = cell_vel_glob.squeeze(0)
        
#         _, (hidden_pose_glob, cell_pose_glob) = self.pose_glob_encoder(pose_glob.permute(1,0,2))
#         hidden_pose_glob = hidden_pose_glob.squeeze(0)
#         cell_pose_glob = cell_pose_glob.squeeze(0)
        
#         glob_inp = pose_glob[:,-1,:]
        
#         hidden_glob = hidden_pose_glob + hidden_vel_glob
#         cell_glob = cell_pose_glob + cell_vel_glob
#         for i in range(self.args.output//self.args.skip):
#             hidden_glob, cell_glob = self.vel_glob_decoder(glob_inp, (hidden_glob, cell_glob))
#             glob_output  = self.fc_global(hidden_glob)
#             glob_outputs = torch.cat((glob_outputs, glob_output.unsqueeze(1)), dim = 1)
#             glob_inp  = glob_output.detach()
       
#         outputs.append(glob_outputs)
        
       
        return tuple(outputs)

class myDataset_DE(torch.utils.data.Dataset):
    def __init__(self, args, dtype):
        
        self.args = args
        self.dtype = dtype
        print("Loading",self.dtype)
        
        sequence_centric = pd.read_csv("sequences_openpifpaf_"+self.dtype+".csv")
#         sequence_centric = pd.read_csv("sequences_4_"+self.dtype+".csv")

        df = sequence_centric.copy()      
        for v in list(df.columns.values):
            print(v+' loaded')
            try:
                df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
            except:
                continue
        sequence_centric[df.columns] = df[df.columns]
        self.data = sequence_centric.copy().reset_index(drop=True)
    
        print('*'*30)
        

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index):

        seq = self.data.iloc[index]
        outputs = []

        obs = torch.tensor([seq.Pose[i] for i in range(0,self.args.input,self.args.skip)])        
        obs_speed = (obs[1:] - obs[:-1])
        
        true = torch.tensor([seq.Future_Pose[i] for i in range(0,self.args.output,self.args.skip)])
        true_speed = torch.cat(((true[0]-obs[-1]).unsqueeze(0), true[1:]-true[:-1]))
        
        outputs.append(obs_speed)
        outputs.append(true_speed)
       
        obs_resh = torch.reshape(obs, (obs.size()[0],17,2))
        
        obs_global=(obs_resh[:,5]+obs_resh[:,6])/2.0
        obs_global=obs_global.unsqueeze(1)

        obs_resh=obs_resh-obs_global
    
        obs_resh=obs_resh.reshape(obs.size())
        
        true_resh = torch.reshape(true, (true.size()[0],17,2))
        true_global=(true_resh[:,5]+true_resh[:,6])/2.0
        true_global=true_global.unsqueeze(1)
        
        true_resh=true_resh-true_global
        true_resh=true_resh.reshape(true.size())
        
        obs_global=torch.reshape(obs_global, (obs.size()[0],2))
        true_global=torch.reshape(true_global, (true.size()[0],2))
        
        obs_global_speed = (obs_global[1:] - obs_global[:-1])
        true_global_speed = torch.cat(((true_global[0]-obs_global[-1]).unsqueeze(0), true_global[1:]-true_global[:-1]))
        

        
        outputs.append(obs_resh)
        outputs.append(true_resh)
        outputs.append(obs_global)
        outputs.append(true_global)
        outputs.append(obs_global_speed)
        outputs.append(true_global_speed)
        outputs.append(true)

        
        return tuple(outputs)    
    
    
def data_loader_DE(args,data):
    dataset = myDataset_DE(args,data)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size, shuffle=args.loader_shuffle,
        pin_memory=args.pin_memory)

    return dataloader

class myDataset(torch.utils.data.Dataset):
    def __init__(self, args, dtype):
        
        self.args = args
        self.dtype = dtype
        print("Loading",self.dtype)
        
        sequence_centric = pd.read_csv("sequences_openpifpaf_"+self.dtype+".csv")
#         sequence_centric = pd.read_csv("sequences_4_"+self.dtype+".csv")

        df = sequence_centric.copy()      
        for v in list(df.columns.values):
            print(v+' loaded')
            try:
                df.loc[:,v] = df.loc[:, v].apply(lambda x: literal_eval(x))
            except:
                continue
        sequence_centric[df.columns] = df[df.columns]
        self.data = sequence_centric.copy().reset_index(drop=True)
    
        print('*'*30)
        

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index):

        seq = self.data.iloc[index]
        outputs = []

#         observed = torch.tensor(np.array(seq.Pose))
#         future = torch.tensor(np.array(seq.Future_Pose))
        
        

        obs = torch.tensor([seq.Pose[i] for i in range(0,self.args.input,self.args.skip)])
        #obs and future global avg of joint 5 & 6 (0 indexed) for openpifpaf
        #local = obs pose-global  for openpifpaf
        
        obs_speed = (obs[1:] - obs[:-1])
    
        outputs.append(obs_speed)
        
        
        true = torch.tensor([seq.Future_Pose[i] for i in range(0,self.args.output,self.args.skip)])
        true_speed = torch.cat(((true[0]-obs[-1]).unsqueeze(0), true[1:]-true[:-1]))


        outputs.append(true_speed)
        outputs.append(obs)
        outputs.append(true)
        
        return tuple(outputs)    
    
    
def data_loader(args,data):
    dataset = myDataset(args,data)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=args.batch_size, shuffle=args.loader_shuffle,
        pin_memory=args.pin_memory)

    return dataloader

import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import numpy as np
import utils
from sklearn.metrics import recall_score, accuracy_score, average_precision_score, precision_score

# import DataLoader
# import network
# import utils

class args():
    def __init__(self):
        self.jaad_dataset = '/data/smailait-data/JAAD/processed_annotations' #folder containing parsed jaad annotations (used when first time loading data)
        self.dtype        = 'train'
        self.from_file    = False #read dataset from csv file or reprocess data
        self.save         = True
        self.file         = '/data/smailait-data/jaad_train_16_16.csv'
        self.save_path    = '/data/smailait-data/jaad_train_16_16.csv'
        self.model_path    = '/data/smailait-data/models/multitask_pv_lstm_trained.pkl'
        self.loader_workers = 1
        self.loader_shuffle = True
        self.pin_memory     = False
        self.image_resize   = [240, 426]
        self.device         = 'cuda'
        self.batch_size     = 50
        self.n_epochs       = 250
        self.hidden_size    = 1000
        self.hardtanh_limit = 100
        self.input  = 16
        self.output = 16
        self.stride = 16
        self.skip   = 1
        # self.task   = 'bounding_box-intention'
        self.task   = 'pose'
        self.use_scenes = False       
        self.lr = 0.03
        
args = args()

net = PV_LSTM_DE_pose(args).to(args.device)

optimizer = optim.Adam(net.parameters(), lr=args.lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=15, 
                                                 threshold = 1e-8, verbose=True)
mse = nn.MSELoss()
bce = nn.BCELoss()
l1e = nn.L1Loss()
train_s_scores = []
train_pose_scores=[]
val_pose_scores=[]
train_c_scores = []
val_s_scores   = []
val_c_scores   = []
train_g_scores = []
val_g_scores   = []

train_loader=data_loader(args,"train")
val_loader=data_loader(args,"val" )

def speed2pos_glob(preds, glob_pred, obs_p, obs_global) :
    pred_pos = torch.zeros(preds.shape[0], preds.shape[1], 34).to('cuda')

    current = (obs_p[:,-1,:].reshape(-1,17,2)+ obs_global[:,-1,:].reshape(-1,1,2)).reshape(-1,34)
    
    for i in range(preds.shape[1]):
        pred_pos[:,i,:] = ((current + preds[:,i,:]).reshape(current.size()[0],17,2)+glob_pred[:,i,:].reshape(-1,1,2)).reshape(-1,34)
        current = pred_pos[:,i,:]
        
    for i in range(34):
        pred_pos[:,:,i] = torch.min(pred_pos[:,:,i], 1920*torch.ones(pred_pos.shape[0], pred_pos.shape[1], device='cuda'))
        pred_pos[:,:,i] = torch.max(pred_pos[:,:,i], torch.zeros(pred_pos.shape[0], pred_pos.shape[1], device='cuda'))
        
    return pred_pos

def ADE_c_DE(pred, true):
    pred = torch.reshape(pred, (pred.size()[0],pred.size()[1],1,2))
    true = torch.reshape(true, (true.size()[0],true.size()[1],1,2))
    
    displacement=torch.sqrt((pred[:,:,:,0]-true[:,:,:,0])**2+(pred[:,:,:,1]-true[:,:,:,1])**2)
    ade = torch.mean(torch.mean(displacement,dim=1))

    return ade


def FDE_c_DE(pred, true):
    pred = torch.reshape(pred, (pred.size()[0],pred.size()[1],1,2))
    true = torch.reshape(true, (true.size()[0],true.size()[1],1,2))
    
    displacement=torch.sqrt((pred[:,-1,:,0]-true[:,-1,:,0])**2+(pred[:,-1,:,1]-true[:,-1,:,1])**2)

    fde = torch.mean(torch.mean(displacement,dim=1))
    
    return fde

def ADE_c(pred, true):
    pred = torch.reshape(pred, (pred.size()[0],pred.size()[1],17,2))
    true = torch.reshape(true, (true.size()[0],true.size()[1],17,2))
    
    displacement=torch.sqrt((pred[:,:,:,0]-true[:,:,:,0])**2+(pred[:,:,:,1]-true[:,:,:,1])**2)
    ade = torch.mean(torch.mean(displacement,dim=1))

    return ade


def FDE_c(pred, true):
    pred = torch.reshape(pred, (pred.size()[0],pred.size()[1],17,2))
    true = torch.reshape(true, (true.size()[0],true.size()[1],17,2))
    
    displacement=torch.sqrt((pred[:,-1,:,0]-true[:,-1,:,0])**2+(pred[:,-1,:,1]-true[:,-1,:,1])**2)

    fde = torch.mean(torch.mean(displacement,dim=1))
    
    return fde

optimizer = optim.Adam(net.parameters(), lr=args.lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=15, 
                                                 threshold = 1e-8, verbose=True)
print('='*100)
print('Training ...')
alpha=0.3

for epoch in range(args.n_epochs):
    start = time.time()
    
    avg_epoch_train_s_loss = 0
    avg_epoch_val_s_loss   = 0
    avg_epoch_train_p_loss   = 0
    avg_epoch_val_p_loss     = 0 
    avg_epoch_train_g_loss   = 0
    avg_epoch_val_g_loss     = 0 
    ade  = 0
    fde  = 0
    ade_train  = 0
    fde_train  = 0
    counter = 0
    
    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(train_loader):
        counter += 1        
        

        
        obs_s    = obs_s.to(device='cuda')
        target_s = target_s.to(device='cuda')
        obs_pose    = obs_pose.to(device='cuda')
        target_pose = target_pose.to(device='cuda')
#         obs_pose_global = obs_pose_global.to(device='cuda')
#         target_pose_global = target_pose_global.to(device='cuda')
#         obs_s_g  = obs_s_g.to(device='cuda')
#         target_s_g = target_s_g.to(device='cuda')
#         target = target.to(device='cuda')
        
#         pose_inp=(obs_pose.reshape(obs_pose.shape[0],obs_pose.shape[1],17,2)+ obs_pose_global.reshape(obs_pose.shape[0],obs_pose.shape[1],1,2)).reshape(obs_pose.shape[0],obs_pose.shape[1],34)
#         vel_inp=(obs_s.reshape(obs_s.shape[0],obs_s.shape[1],17,2)+ obs_s_g.reshape(obs_s.shape[0],obs_s.shape[1],1,2)).reshape(obs_s.shape[0],obs_s.shape[1],34)
        
#         print(pose_inp.size())
#         print(vel_inp.size())
        
        net.zero_grad()
    
#         (speed_preds,global_preds) = net(pose=obs_pose, vel=obs_s, pose_glob=obs_pose_global, vel_glob=obs_s_g)
        (preds_p,) = net(pose=obs_pose, vel=obs_s )
#         print(preds_p.size())

#         speed_loss  = mse(speed_preds, target_s)
#         global_loss =  mse(global_preds, target_s_g)
#         speed_loss  = l1e(speed_preds, target_s)
#         global_loss =  l1e(global_preds, target_s_g)
        
#         loss = (1-alpha)*speed_loss + alpha*global_loss
        loss=mse(preds_p,target_pose)
        loss.backward()
    
       
    
        ade_train += float(ADE_c(preds_p, target_pose))
        fde_train += float(FDE_c(preds_p, target_pose))
        
        optimizer.step()
        
#         avg_epoch_train_s_loss += float(speed_loss)
#         avg_epoch_train_g_loss += float(global_loss)
        
    avg_epoch_train_s_loss /= counter
    avg_epoch_train_g_loss /= counter
    train_s_scores.append(avg_epoch_train_s_loss)
    train_g_scores.append(avg_epoch_train_g_loss)
    ade_train  /= counter
    fde_train  /= counter  
    counter=0

    for idx, (obs_s, target_s, obs_pose, target_pose) in enumerate(val_loader):
        counter+=1
        obs_s    = obs_s.to(device='cuda')
        target_s = target_s.to(device='cuda')
        obs_pose    = obs_pose.to(device='cuda')
        target_pose = target_pose.to(device='cuda')
#         obs_pose_global = obs_pose_global.to(device='cuda')
#         target_pose_global = target_pose_global.to(device='cuda')
#         obs_s_g  = obs_s_g.to(device='cuda')
#         target_s_g = target_s_g.to(device='cuda')
#         target = target.to(device='cuda')
        
#         print(obs_pose[0,0])
        
#         pose_inp=(obs_pose.reshape(obs_pose.shape[0],obs_pose.shape[1],17,2)+ obs_pose_global.reshape(obs_pose.shape[0],obs_pose.shape[1],1,2)).reshape(obs_pose.shape[0],obs_pose.shape[1],34)
#         vel_inp=(obs_s.reshape(obs_s.shape[0],obs_s.shape[1],17,2)+ obs_s_g.reshape(obs_s.shape[0],obs_s.shape[1],1,2)).reshape(obs_s.shape[0],obs_s.shape[1],34)
        
       
        
        with torch.no_grad():
#             (speed_preds,global_preds) = net(pose=obs_pose, vel=obs_s, pose_glob=obs_pose_global, vel_glob=obs_s_g)

#             speed_loss  = mse(speed_preds, target_s)
#             global_loss =  mse(global_preds, target_s_g)
#             loss = (1-alpha)*speed_loss + alpha*global_loss
#             avg_epoch_val_s_loss += float(speed_loss)
#             avg_epoch_val_g_loss += float(global_loss)
            
#             preds_p = speed2pos_glob(speed_preds, global_preds, obs_pose, obs_pose_global) 
            
            (preds_p,) = net(pose=obs_pose, vel=obs_s)

            loss=mse(preds_p,target_pose)
            
            ade += float(ADE_c(preds_p, target_pose))
            fde += float(FDE_c(preds_p, target_pose))

        
    avg_epoch_val_s_loss /= counter
    avg_epoch_val_g_loss /= counter
    val_s_scores.append(avg_epoch_val_s_loss)
    val_g_scores.append(avg_epoch_val_g_loss)
    
    ade  /= counter
    fde  /= counter     
    
    scheduler.step(loss)
    
     
    print('e:', epoch, '| ts: %.4f'% avg_epoch_train_s_loss, '| tg: %.4f'% avg_epoch_train_g_loss, '| vs: %.4f'% avg_epoch_val_s_loss, '| vg: %.4f'% avg_epoch_val_g_loss, '| ade_train: %.4f'% ade_train, '| ade_val: %.4f'% ade, '| fde_train: %.4f'% fde_train,'| fde_val: %.4f'% fde)
#           ,'| t:%.4f'%(time.time()-start))


print('='*100) 
# print('Saving ...')
# torch.save(net.state_dict(), args.model_path)
print('Done !')

Loading train
Pose loaded
Future_Pose loaded
******************************
Loading val
Pose loaded
Future_Pose loaded
******************************
Training ...
e: 0 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 49.6210 | ade_val: 45.8807 | fde_train: 99.0719 | fde_val: 84.3724
e: 1 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 36.7938 | ade_val: 43.1043 | fde_train: 69.9444 | fde_val: 77.9061
e: 2 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 46.2328 | ade_val: 52.1490 | fde_train: 93.4079 | fde_val: 105.9055
e: 3 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 41.7121 | ade_val: 54.3765 | fde_train: 85.0027 | fde_val: 115.4621
e: 4 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 46.6465 | ade_val: 53.0824 | fde_train: 102.7775 | fde_val: 119.0059
e: 5 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 42.2438 | ade_val: 47.8939 | fde_train: 96.4135 | fde_val: 106.1895
e: 6 | ts:

e: 58 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 11.3166 | ade_val: 18.9716 | fde_train: 17.3780 | fde_val: 35.0073
e: 59 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 11.2448 | ade_val: 17.8731 | fde_train: 17.9314 | fde_val: 31.9117
e: 60 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 11.2371 | ade_val: 19.5619 | fde_train: 17.1567 | fde_val: 37.8364
e: 61 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 11.9516 | ade_val: 17.1256 | fde_train: 20.2845 | fde_val: 30.2833
e: 62 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 11.5239 | ade_val: 17.6262 | fde_train: 17.5982 | fde_val: 32.5411
e: 63 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 11.5254 | ade_val: 17.7185 | fde_train: 18.0135 | fde_val: 33.0944
e: 64 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 11.6427 | ade_val: 17.5518 | fde_train: 18.5846 | fde_val: 31.6116
Epoch    66: reducing learning rat

e: 116 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.7959 | ade_val: 18.3443 | fde_train: 11.6857 | fde_val: 33.3674
e: 117 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.7813 | ade_val: 18.4000 | fde_train: 11.5159 | fde_val: 33.4073
e: 118 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.7644 | ade_val: 18.3886 | fde_train: 11.3549 | fde_val: 33.4002
e: 119 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.7429 | ade_val: 18.4324 | fde_train: 11.3797 | fde_val: 33.4466
e: 120 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.7179 | ade_val: 18.4260 | fde_train: 11.3509 | fde_val: 33.4308
e: 121 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.7060 | ade_val: 18.3956 | fde_train: 11.2767 | fde_val: 33.4406
e: 122 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.6700 | ade_val: 18.3724 | fde_train: 11.2119 | fde_val: 33.4590
e: 123 | ts: 0.0000 | tg: 0.0000 |

e: 175 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3786 | ade_val: 17.6601 | fde_train: 10.7130 | fde_val: 32.5663
e: 176 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3843 | ade_val: 17.6444 | fde_train: 10.6983 | fde_val: 32.5628
Epoch   178: reducing learning rate of group 0 to 1.1719e-04.
e: 177 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3593 | ade_val: 17.6349 | fde_train: 10.6752 | fde_val: 32.5749
e: 178 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3660 | ade_val: 17.6304 | fde_train: 10.6965 | fde_val: 32.5696
e: 179 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3667 | ade_val: 17.6268 | fde_train: 10.6921 | fde_val: 32.5610
e: 180 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3619 | ade_val: 17.6220 | fde_train: 10.6586 | fde_val: 32.5483
e: 181 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3642 | ade_val: 17.6175 | fde_train:

e: 233 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3256 | ade_val: 17.8730 | fde_train: 10.5802 | fde_val: 32.9401
e: 234 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3300 | ade_val: 17.8722 | fde_train: 10.5925 | fde_val: 32.9379
e: 235 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3292 | ade_val: 17.8708 | fde_train: 10.5740 | fde_val: 32.9355
e: 236 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3347 | ade_val: 17.8683 | fde_train: 10.5826 | fde_val: 32.9321
e: 237 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3278 | ade_val: 17.8656 | fde_train: 10.5758 | fde_val: 32.9292
e: 238 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3268 | ade_val: 17.8592 | fde_train: 10.5777 | fde_val: 32.9196
e: 239 | ts: 0.0000 | tg: 0.0000 | vs: 0.0000 | vg: 0.0000 | ade_train: 8.3374 | ade_val: 17.8450 | fde_train: 10.5984 | fde_val: 32.8690
e: 240 | ts: 0.0000 | tg: 0.0000 |

In [ ]:
seq_out=np.delete(array_of_seq, list(range(2, array_of_seq.shape[3], 3)), axis=3)

In [1]:
%cd Sc-sfmlearner

C:\Users\bigbo\Desktop\College\Project\Sc-sfmlearner


In [2]:
import torch

from imageio import imread, imsave
from skimage.transform import resize as imresize
import matplotlib.pyplot as plt

import numpy as np

import argparse
from tqdm import tqdm
import os 

from models import DispResNet
from utils2 import tensor2array
import glob


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
img_height=256
img_width=832
img_exts=['png', 'jpg', 'bmp']

resnet_layers=18
output_dir='results/'
pretrained='checkpoints/resnet18_depth_256/dispnet_model_best.pth.tar'

with torch.no_grad():

    # args = parser.parse_args()
    # if not(args.output_disp or args.output_depth):
    #     print('You must at least output one value !')
    #     return

    disp_net = DispResNet(resnet_layers, False).to(device)
    weights = torch.load(pretrained)
    disp_net.load_state_dict(weights['state_dict'])
    disp_net.eval()

#     dataset_dir = Path(dataset_dir)
#     output_dir = Path(output_dir)
#     output_dir.mkdir()

#     test_files = sum([dataset_dir.files('*.{}'.format(ext)) for ext in img_exts], [])
    test_files=glob.glob('./../frames/*/*.png')
    

    print('{} files to test'.format(len(test_files)))

    for file in tqdm(test_files):
#         print(file)
        dst='results'+file.replace('./../frames','').replace('.png','')
#         print(dst)
#         break
        if not os.path.exists(dst+'_disp.png'):
#         print(dst)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
    #         print(file)

            img = imread(file).astype(np.float32)

            h, w, _ = img.shape

            img = imresize(img, (img_height, img_width)).astype(np.float32)

    #         fig = plt.figure(figsize=(15, 15))
    #         a = fig.add_subplot(1, 3, 1)
    #         imgplot = plt.imshow(img.astype(np.uint8))

            img = np.transpose(img, (2, 0, 1))

            tensor_img = torch.from_numpy(img).unsqueeze(0)
            tensor_img = ((tensor_img/255 - 0.45)/0.225).to(device)

            output = disp_net(tensor_img)[0]


            disp = (255*tensor2array(output, max_value=None, colormap='bone')).astype(np.uint8)


            depth = 1/output
            depth = (255*tensor2array(depth, max_value=1, colormap='magma')).astype(np.uint8)

    #         a = fig.add_subplot(1, 3, 2)
    #         imgplot = plt.imshow(np.transpose(disp,(1,2,0)))
    #         a = fig.add_subplot(1, 3, 3)
    #         imgplot = plt.imshow(np.transpose(depth,(1,2,0)))



            imsave(dst+'_disp.png', np.transpose(disp, (1, 2, 0)))
            imsave(dst+'_depth.png', np.transpose(depth, (1, 2, 0)))




  4%|██▋                                                                         | 592/16819 [00:00<00:02, 5897.73it/s]

16819 files to test


100%|██████████████████████████████████████████████████████████████████████████| 16819/16819 [00:02<00:00, 6670.96it/s]


In [2]:
from inverse_warp import pose_vec2mat
from scipy.ndimage.interpolation import zoom
from inverse_warp import *
import models

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
sequence='test'
pretrained_posenet='./checkpoints/resnet50_pose_256/exp_pose_model_best.pth.tar'
def load_tensor_image(filename):
    img = imread(filename).astype(np.float32)
    h, w, _ = img.shape
    # if (not args.no_resize) and (h != img_height or w != img_width):
    img = imresize(img, (img_height, img_width)).astype(np.float32)
    img = np.transpose(img, (2, 0, 1))
    tensor_img = ((torch.from_numpy(img).unsqueeze(0)/255-0.45)/0.225).to(device)
    return tensor_img


with torch.no_grad():

    weights_pose = torch.load(pretrained_posenet)
    pose_net = models.PoseResNet().to(device)
    pose_net.load_state_dict(weights_pose['state_dict'], strict=False)
    pose_net.eval()

    image_dir = Path(dataset_dir)
    output_dir = Path('./')
    output_dir.makedirs_p()

    test_files = sum([image_dir.files('*.{}'.format(ext)) for ext in img_exts], [])
    test_files.sort()

    print('{} files to test'.format(len(test_files)))

    global_pose = np.eye(4)
    poses = [global_pose[0:3, :].reshape(1, 12)]

    n = len(test_files)
    tensor_img1 = load_tensor_image(test_files[0])

    for iter in tqdm(range(n - 1)):

        tensor_img2 = load_tensor_image(test_files[iter+1])

        pose = pose_net(tensor_img1, tensor_img2)

        pose_mat = pose_vec2mat(pose).squeeze(0).cpu().numpy()
        pose_mat = np.vstack([pose_mat, np.array([0, 0, 0, 1])])
        global_pose = global_pose @  np.linalg.inv(pose_mat)

        poses.append(global_pose[0:3, :].reshape(1, 12))

        # update
        tensor_img1 = tensor_img2

    poses = np.concatenate(poses, axis=0)
    filename = Path(output_dir + sequence + ".txt")
    np.savetxt(filename, poses, delimiter=' ', fmt='%1.8e')




NameError: name 'Path' is not defined